In [1]:
from keras_transformer import *

import numpy as np
from keras_layer_normalization import LayerNormalization
from keras_multi_head import MultiHeadAttention
from keras_position_wise_feed_forward import FeedForward
from keras_pos_embd import TrigPosEmbedding
from keras_embed_sim import EmbeddingRet, EmbeddingSim
import keras
from keras.layers import Dense,Reshape

def get_m(token_num,
              embed_dim,
              decoder_num,
              head_num,
              hidden_dim,
              attention_activation=None,
              feed_forward_activation='relu',
              dropout_rate=0.0,
              embed_weights =None,
              embed_trainable=None,
              trainable=True,
              use_adapter=False,
              adapter_units=None,
              adapter_activation='relu'):
    """Get full model without compilation.
    :param token_num: Number of distinct tokens.
    :param embed_dim: Dimension of token embedding.
    :param encoder_num: Number of encoder components.
    :param decoder_num: Number of decoder components.
    :param head_num: Number of heads in multi-head self-attention.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param attention_activation: Activation for multi-head self-attention.
    :param feed_forward_activation: Activation for feed-forward layer.
    :param dropout_rate: Dropout rate.
    :param use_same_embed: Whether to use the same token embedding layer. `token_num`, `embed_weights` and
                           `embed_trainable` should be lists of two elements if it is False.
    :param embed_weights: Initial weights of token embedding.
    :param embed_trainable: Whether the token embedding is trainable. It will automatically set to False if the given
                            value is None when embedding weights has been provided.
    :param trainable: Whether the layers are trainable.
    :param use_adapter: Whether to use feed-forward adapters before each residual connections.
    :param adapter_units: The dimension of the first transformation in feed-forward adapter.
    :param adapter_activation: The activation after the first transformation in feed-forward adapter.
    :return: Keras model.
    """
    decoder_token_num = token_num

    decoder_embed_weights = embed_weights

    if decoder_embed_weights is not None:
        decoder_embed_weights = [decoder_embed_weights]

    decoder_embed_trainable = embed_trainable

    if decoder_embed_trainable is None:
        decoder_embed_trainable = decoder_embed_weights is None


    decoder_embed_layer = EmbeddingRet(
        input_dim=decoder_token_num,
        output_dim=embed_dim,
        mask_zero=True,
        weights=decoder_embed_weights,
        trainable=decoder_embed_trainable,
        name='Decoder-Token-Embedding',
    )
    encoder_output = keras.layers.Input(shape=(1,512), name='Encoder-Output',dtype='float32')
    encoder_output_divider1 = Dense(512,input_shape=(None,1,512), name='Encoder-Output-Divider-1',activation='tanh')(encoder_output)
    #divider_reshape1=Reshape(target_shape=(1,512),name='Divider-Reshape-1')(encoder_output_divider1)
    encoder_output_divider2 = Dense(512,input_shape=(None,1,512), name='Encoder-Output-Divider-2',activation='relu')(encoder_output_divider1)#(divider_reshape1)
    #divider_reshape2=Reshape(target_shape=(1,512),name='Divider-Reshape-2')(encoder_output_divider2)
    decoder_input = keras.layers.Input(shape=(None,), name='Decoder-Input')
    decoder_embed, decoder_embed_weights = decoder_embed_layer(decoder_input)
    decoder_embed = TrigPosEmbedding(
        mode=TrigPosEmbedding.MODE_ADD,
        name='Decoder-Embedding',
    )(decoder_embed)
    decoded_layer = get_decoders(
        decoder_num=decoder_num,
        input_layer=decoder_embed,
        encoded_layer=encoder_output_divider2,#divider_reshape2,#encoder_output,
        head_num=head_num,
        hidden_dim=hidden_dim,
        attention_activation=attention_activation,
        feed_forward_activation=feed_forward_activation,
        dropout_rate=dropout_rate,
        trainable=trainable,
        use_adapter=use_adapter,
        adapter_units=adapter_units,
        adapter_activation=adapter_activation,
    )
    dense_layer = EmbeddingSim(
        trainable=trainable,
        name='Output',
    )([decoded_layer, decoder_embed_weights])
    return keras.models.Model(inputs=[encoder_output, decoder_input], outputs=dense_layer)

Using TensorFlow backend.
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\dec2\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Fu

In [2]:
def piecesextender(data):
    kol= np.zeros(shape=(data.shape[0]))
    for i in range(data.shape[0]):
        kol[i]=np.count_nonzero(data[i])
    kol = kol.astype(np.int)
    out = np.zeros(shape=(np.sum(kol),512))
    buf = 0
    for g in range(data.shape[0]):        
        for k in range(1,kol[g]):
            out[buf][0]=3000
            out[buf][1:513-k]+=data[g][k:]
            buf+=1
    return out, kol

def vectorsextender(data,kol):
    out = np.zeros(shape=(np.sum(kol),512))
    buf=0
    for g in range(data.shape[0]):
        for k in range(kol[g]-1):
            out[buf]+=data[g]
            buf+=1
    return out

In [3]:
from scipy.ndimage.interpolation import shift

model = get_m(
    token_num=3003,
    embed_dim=512,
    decoder_num=4,
    head_num=4,
    hidden_dim=120,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((3003, 512)),
)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Decoder-Token-Embedding (Embedd [(None, None, 512),  1537536     Decoder-Input[0][0]              
__________________________________________________________________________________________________
Decoder-Embedding (TrigPosEmbed (None, None, 512)    0           Decoder-Token-Embedding[0][0]    
__________________________________________________________________________________________________
Decoder-1-MultiHeadSelfAttentio (None, None, 512)    1050624     Decod

In [3]:
from keras.models import load_model
model = load_model ('D:/decoder/models/m5.h5' , custom_objects = get_custom_objects ())






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
from scipy.ndimage.interpolation import shift
n=25
e=26

for i in range(5,1000):
    vectors_train = np.load('D:\\decoder\\data\\data3000\\news_vectors'+str(i)+'.npy')
    pieces_train = np.load('D:\\decoder\\data\\data3000\\news_pieces'+str(i)+'.npy')
    shifting = np.ones(shape=(pieces_train.shape[0],512))
    pieces_train = pieces_train+shifting
    
    print('Readed')
    '''
    ptt, kol = piecesextender(pieces_train)
    shifting = np.ones(shape=(ptt.shape[0],512))
    ptt = ptt + shifting
    print('Pt converted')
    pot = np.copy(ptt)
    pot = shift(pot, (0,-1))
    pot = pot.reshape(pot.shape[0],512,1)
    print('Po converted')
    vtt = vectorsextender(vectors_train, kol)
    print('V converted')

    model.fit(
        x=[vtt.reshape(vtt.shape[0],1,512), 
           ptt],
        y = pot,
        epochs=i, initial_epoch=i+1,
        validation_split=0.05,
        #batch_size=1
    )
    '''
        
    for j in range(5):
        ptt, kol = piecesextender(pieces_train[0+j*100:(1+j)*100])
        #shifting = np.ones(shape=(ptt.shape[0],512))
        #ptt = ptt + shifting
        #print('Pt converted')
        pot = np.copy(ptt)
        pot = shift(pot, (0,-1))
        pot = pot.reshape(pot.shape[0],512,1)
        #print('Po converted')
        vtt = vectorsextender(vectors_train[0+j*100:(1+j)*100], kol)
        #print('V converted')
        
        
        model.fit(
            x=[vtt.reshape(vtt.shape[0],1,512), 
               ptt],
            y = pot,
            epochs=e, initial_epoch=n,
            validation_split=0.05,
            batch_size=4
        )
        n+=1
        e+=1

    if i%10==0:
        model.save('D:\\decoder\\models\\m'+str(i)+'.h5')

Readed
Train on 5318 samples, validate on 280 samples
Epoch 26/26
5318/5318 [==============================] - 316s 59ms/step - loss: 2.0357 - val_loss: 3.9100
Train on 5083 samples, validate on 268 samples
Epoch 27/27
5083/5083 [==============================] - 295s 58ms/step - loss: 1.8933 - val_loss: 1.6678
Train on 5615 samples, validate on 296 samples
Epoch 28/28
5615/5615 [==============================] - 327s 58ms/step - loss: 1.7650 - val_loss: 2.4289
Train on 4158 samples, validate on 219 samples
Epoch 29/29
4158/4158 [==============================] - 240s 58ms/step - loss: 1.9302 - val_loss: 2.3757
Train on 5320 samples, validate on 281 samples
Epoch 30/30
5320/5320 [==============================] - 306s 58ms/step - loss: 1.9759 - val_loss: 5.6564
Readed
Train on 5318 samples, validate on 280 samples
Epoch 31/31
5318/5318 [==============================] - 307s 58ms/step - loss: 1.7515 - val_loss: 3.7778
Train on 5083 samples, validate on 268 samples
Epoch 32/32
5083/5083

KeyboardInterrupt: 

In [7]:
model.save('D:\\decoder\\models\\m5.h5')

In [5]:
model.save('C:\\Users\\DNS\\Desktop\\Progi\\Python\\NeuralNetworks\\Курсач\\models\\m5.h5')